In [1]:
import polars as pl
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2

In [2]:
#df = pl.read_csv('Final Transactions.csv')
#df = df[["TX_DATETIME", "CUSTOMER_ID", "TERMINAL_ID", "TX_AMOUNT", "TX_FRAUD", "TX_FRAUD_SCENARIO"]]
#df = df.to_pandas()
df = pd.read_csv("Final Transactions.csv")
df = df[["TX_DATETIME", "CUSTOMER_ID", "TERMINAL_ID", "TX_AMOUNT", "TX_FRAUD", "TX_FRAUD_SCENARIO"]]

In [3]:
df.columns = [x.lower() for x in ["TX_DATETIME", "CUSTOMER_ID", "TERMINAL_ID", "TX_AMOUNT", "TX_FRAUD", "TX_FRAUD_SCENARIO"]]
df["tx_datetime"] = pd.to_datetime(df['tx_datetime'])

In [4]:
pg_user = 'postgres'
pg_password = 'a5TKQ7SSVGFuQIz5'
pg_hostname = 'primeds.protal.biz:30308'
pg_db_name = 'postgres'

pg_engine = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_hostname}/{pg_db_name}')
pg_conn = pg_engine.connect()

In [5]:
table_query = '''CREATE TABLE IF NOT EXISTS staging.tx_fraud (
  tx_datetime                   TIMESTAMPTZ       NOT NULL,
  customer_id                   DOUBLE PRECISION  NULL,
  terminal_id                   DOUBLE PRECISION  NULL,
  tx_amount                     DOUBLE PRECISION  NULL,
  tx_fraud                      DOUBLE PRECISION  NULL,
  tx_fraud_scenario             DOUBLE PRECISION  NULL
);'''
result = pg_conn.execute(text(table_query))
pg_conn.commit()

In [6]:
hypertable_query = '''SELECT create_hypertable('staging.tx_fraud', 'tx_datetime');'''
result = pg_conn.execute(text(hypertable_query))

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "timestamp" does not exist

[SQL: SELECT create_hypertable('staging.tx_fraud', 'timestamp');]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
df.to_sql(name="tx_fraud", con=pg_conn, index=False, if_exists='append', schema='staging')
pg_conn.commit()
pg_conn.close()